In [1]:
# Imports
import pandas as pd
import lib.SAAT as SAAT
import lib.utils as utils

In [2]:
# Read & Clean
data = pd.read_csv("data/example_data.csv")

# Get department from email
data["Department"] = data["Email"].map(lambda email: utils.email_to_department(email))
data = data[data['Topic'] == 'Topic1']

data.head()

,Name,Email,Years_In_Org,Grade,Setup,Topic,1-2pm,2-3pm,3-4pm,4-5pm,...,2-3pm2,3-4pm2,4-5pm2,5-6pm2,1-2pm.1,2-3pm3,3-4pm3,4-5pm3,5-6pm3,Department
0,Person1,Person1@DepartmentG.com,<1 year,Level1,If needed,Topic1,0,0,1,1,...,0,1,1,1,0,0,1,1,1,departmentg
9,Person10,Person10@DepartmentF.com,1-3 years,Level2,If needed,Topic1,1,1,0,0,...,0,0,0,0,0,1,1,0,0,departmentf
11,Person12,Person12@DepartmentH.com,5-10 years,Level2,Yes,Topic1,0,0,0,1,...,1,1,1,1,1,1,1,1,0,departmenth
15,Person16,Person16@DepartmentL.com,1-3 years,Level2,If needed,Topic1,0,0,0,1,...,0,0,1,1,0,0,0,0,0,departmentl
16,Person17,Person17@DepartmentG.com,>10 years,Level2,No,Topic1,1,1,1,0,...,1,1,0,0,0,1,0,1,0,departmentg


In [3]:
# Initialise SAAT
my_SAAT = SAAT.SAAT(
    times = data.loc[:,'1-2pm':'5-6pm3'], 
    group_size = 4, 
    organise = data["Setup"], 
    department = data["Department"], 
    time_in_company = data["Years_In_Org"])

my_SAAT.set_schedule(my_SAAT.auto(minutes=0.1, steps=200))
my_SAAT.copy_strategy = "slice" # since our state is just a list, slice is the fastest way to copy

#Run
state, e = my_SAAT.anneal()

print('Energy maximised at: {}'.format(-e))
print('Final state: {}'.format(state))

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.05800        -20.97    19.50%     0.00%     0:00:04     0:00:54 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.05800        -20.61    17.31%     0.00%     0:00:07     0:00:00

Energy maximised at: 20.965587138863
Final state: [4, 1, 11, 3, 18, 6, 12, 21, 10, 17, 19, 16, 8, 5, 20, 13, 22, 7, 14, 23, 2, 15, 9]


In [4]:
# Print results
print('Allocation: ' + str(state))
print('Energy maximised at: ' + str(-e))
utils.print_pretty_allocations(
    state = state, 
    times = data.loc[:,'1-2pm':'5-6pm3'], 
    group_size = 4,
    extras = data.iloc[:,2:6])
utils.results_to_csv(
    state = state,
    data = data,
    group_size = 4,
    name='results/Topic_1_Allocation.csv')

Allocation: [4, 1, 11, 3, 18, 6, 12, 21, 10, 17, 19, 16, 8, 5, 20, 13, 22, 7, 14, 23, 2, 15, 9]
Energy maximised at: 20.965587138863

4  0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 ---- 1-3 years ---- Level2 ---- If needed ---- Topic1
1  0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 ---- <1 year ---- Level1 ---- If needed ---- Topic1
11 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 ---- >10 years ---- Level2 ---- Yes ---- Topic1
3  0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 ---- 5-10 years ---- Level2 ---- Yes ---- Topic1

18 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 ---- <1 year ---- Level1 ---- If needed ---- Topic1
6  1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 ---- >10 years ---- Level3 ---- Yes ---- Topic1
12 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 ---- 1-3 years ---- Level2 ---- If needed ---- Topic1
21 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 ---- <1 year ---- Level1 ---- If needed ---- Topic1

10 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ---- >10 years ---- Level3 ---- No ---- Topic1
17 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 ---- <1 year ---- Level2 ---- If needed ---- Topic1
19 0 1 1 1 0 0 1 1 1 0 0 1